In [1]:
import scipy.io as sio
import pandas as pd
import numpy as np

In [2]:
data_mat = sio.loadmat('exam_data/data-qp.mat')
# check keys
data_mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'A', 'C', 'b', 'd'])

In [3]:
A = data_mat['A']
C = data_mat['C']
b = data_mat['b']
d = data_mat['d']

In [117]:
def is_positive_semi_def(x):
    if np.all(np.linalg.eigvals(x) > 0):
        print('positive definite')
    elif np.all(np.linalg.eigvals(x) >= 0):
        print('positive semi definite')
    else:
        print('general matrix')
    # print('eigen values:\n',np.linalg.eigvals(x))

In [118]:
alpha = 1/2
m = A.shape[0]
n = A.shape[1]

In [119]:
# P
P_A = 2*(A.T @ A)
P_C = 2*(C.T @ C)
P = np.concatenate(( np.concatenate((P_A, np.zeros_like(P_A)), axis=1),np.concatenate((np.zeros_like(P_C),P_C), axis=1) ), axis=0)
P = np.concatenate((P, np.zeros((n+1,2*n))), axis=0)     
P = np.concatenate((P, np.zeros((3*n+1,n+1))), axis=1)            
P.shape

(31, 31)

In [120]:
is_positive_semi_def(P)

positive semi definite


In [121]:
# q
q = np.concatenate(( -2*(A.T @ b) ,-2*(C.T @ d) ),axis=0)
q = np.concatenate((q,np.zeros((n+1,1))), axis=0)
q = q.reshape(q.shape[0],)
q.shape

(31,)

In [122]:
# G
G = np.concatenate((np.zeros((1,2*n)),np.ones((1,n+1))),axis=1)
G = np.concatenate((G,
        np.concatenate((np.zeros((n,n)), -1*np.identity(n),-1*np.identity(n), np.zeros((n,1))),axis=1)
        ),axis=0)
G = np.concatenate((G,
        np.concatenate((np.zeros((n,n)),  1*np.identity(n),-1*np.identity(n), np.zeros((n,1))),axis=1)
        ),axis=0)
G = np.concatenate((G,
        np.concatenate((-1*np.identity(n), np.zeros((n,n)), np.zeros((n,n)), -1*np.ones((n,1))),axis=1)
        ),axis=0)
G = np.concatenate((G,
        np.concatenate((1*np.identity(n), np.zeros((n,n)), np.zeros((n,n)), -1*np.ones((n,1))),axis=1)
        ),axis=0)
G.shape

(41, 31)

In [123]:
# h
h = np.concatenate((np.array([alpha]).reshape(1,1),np.zeros((4*n,1))), axis=0)
h = h.reshape(h.shape[0],)
h.shape

(41,)

In [124]:
# M
M = np.zeros((3*n+1,))
M.shape

(31,)

In [125]:
# g
g = np.array([0])

In [127]:
def f(x):
    return (1/2 * x.T @ P @ x) + (q.T @ x)

In [147]:
from cvxopt import matrix, solvers
sol=solvers.qp(matrix(P), matrix(q), matrix(G), matrix(h))
w = np.array(sol['x'])
p_opt = float(f(w))

In [190]:
# result
x = w[0:n]
z = w[n:2*n]
print('Optimal value:', p_opt)
print('{:5}{:15} {:15}'.format('','x', 'z'))
for i in range(n):
    print('{:>12.5E} {:15.5E}'.format(x[i][0], z[i][0]))


Optimal value: -8.500918794739846
     x               z              
 1.21633E-01    -2.59405E-09
 9.14190E-02     1.60422E-10
 2.23835E-01    -5.09069E-09
 5.01055E-02     6.49523E-10
 2.23835E-01    -5.26103E-02
-2.23835E-01    -2.73284E-11
-1.90577E-02    -3.18965E-10
 1.55831E-01    -2.73922E-02
 6.52569E-02     1.13630E-10
-9.95057E-02    -1.96163E-01
